## Cal-CRAI metric: # of cold days
This notebook generates the text metadata files for the extreme heat exposure metric: `change in average number of cold days` using Cal-Adapt: Analytics Engine data.  Please see the processing notebook `climate_ae_cold_days.ipynb` for full methodological process. Note: this notebook can only be run on the AE Jupyter Hub, or a computing environment with a large enough memory capacity (e.g., at least 30 GB).

### Step 1: Generate metadata

In [1]:
import pandas as pd
import os
import sys

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import upload_csv_aws, pull_csv_from_directory
from scripts.utils.write_metadata import append_metadata

In [ ]:
bucket_name = 'ca-climate-index'
aws_dir = '3_fair_data/index_data/climate_heat_cold_days_metric.csv'
folder = 'csv_folder'

pull_csv_from_directory(bucket_name, aws_dir, folder, search_zipped=False)

In [ ]:
df_in = pd.read_csv(r'csv_folder/climate_heat_cold_days_metric.csv') # make sure this is in the same folder!
df_in # check

In [4]:
# Move a specific column to the end of the DataFrame
column_to_move = 'mean_change_cold_days'  # Replace with the actual column name
columns = [col for col in df_in.columns if col != column_to_move]  # Keep all other columns
columns.append(column_to_move)  # Add the column to move to the end

# Reassign the DataFrame with the new column order
df_in = df_in[columns]

In [5]:
df_in.to_csv('climate_heat_cold_days_metric.csv', index=False)

In [10]:
#@append_metadata
def extreme_heat_cold_days_process(df, export=False, export_filename=None, varname=''):
    '''
    Reduces the size of the initial daily raw temperature data in order to streamline compute time.
    Transforms the raw data into the following baseline metrics:
    * change in number of average annual seasonal chill hours
    
    Methods
    -------
    Metric is calculated with 2nd precentile.
    
    Parameters
    ----------
    df: pd.DataFrame
        Input data.
    export: True/False boolean
        False = will not upload resulting df containing CAL CRAI extreme heat metric to AWS
        True = will upload resulting df containing CAL CRAI extreme heat metric to AWS
    export_filename: string
        name of csv file to be uploaded to AWS
    varname: string
        Final metric name, for metadata generation
        
    Script
    ------
    Metric calculation: climate_ae_cold_days.py via pcluster run
    Metadata generation: climate_ae_cold_days_metadata.ipynb
        
    Note
    ----
    Because the climate projections data is on the order of 2.4 TB in size, intermediary
    processed files are not produced for each stage of the metric calculation. All processing
    occurs in a single complete run in the notebook listed above.
    '''
        
    # calculate chronic with 2°C WL
    print('Data transformation: raw projections data retrieved for warming level of 2.0°C, by manually subsetting based on GWL for parent GCM and calculating 30 year average.')
    print("Data transformation: dynamically-downscaled climate data subsetted for a-priori bias-corrected models.")
    print("Data transformation: drop all singleton dimensions (scenario).")
    print("Data transformation: number of cold days calculated by summing the number of days per year below 2nd percentile.")

    # historical baseline
    print("Data transformation: historical baseline data retrieved for 1981-2010, averaging across models.")
    print("Data transformation: dynamically-downscaled climate data subsetted for a-priori bias-corrected models.")
    print("Data transformation: drop all singleton dimensions (scenario).")
    print("Data transformation: number of cold days calculated by summing the number of days per year below 2nd percentile.")

    # calculate delta signal
    print("Data transformation: delta signal calculated by taking difference between chronic (2.0°C) and historical baseline.")

    # reprojection to census tracts
    print("Data transformation: data transformed from xarray dataset into pandas dataframe.")
    print("Data transformation: data reprojected from Lambert Conformal Conic CRS to CRS 3857.")
        
    # min-max standardization
    print("Data transformation: data min-max standardized with min_max_standardize function.")
    
    if export == True:
        bucket_name = 'ca-climate-index'
        directory = '3_fair_data/index_data'
        export_filename = [df]
        upload_csv_aws(export_filename, bucket_name, directory)

    if export == False:
        print(f'{df} uplaoded to AWS.')

    if os.path.exists(df):
        os.remove(df)

In [ ]:
varname = 'climate_heat_cold_days'
filename = 'climate_heat_cold_days_metric.csv'
extreme_heat_cold_days_process(filename, export=True, export_filename=None, varname='test')